# **LLM, RAG 활용한 AI Chat Bot 구현**

- LLM, RAG 기술 활용하여 사용자 질문에 답변하는 챗봇 구현.
- PDF 형식의 문서를 불러와 RAG 구축.

- 활용 데이터 : [한권으로 끝내는 주식과 세금](./한권으로%20끝내는%20주식과%20세금.pdf)

- 여분 데이터(코드 수정으로 변형 가능) :  
[인공지능산업최신동향](./인공지능산업최신동향.pdf)  
[초거대언어모델연구동향](./초거대%20언어모델%20연구%20동향.pdf)  

## *사용 환경 준비*

- 환경변수 파일이나 환경변수 설정을 통해 API key 로드.

In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
from dotenv import load_dotenv

# 환경변수 파일 로드
load_dotenv(dotenv_path='seok.env')

# api key 변수에 환경변수 파일의 api key 저장
api_key = os.getenv("API_KEY")

# 모델 초기화
model = ChatOpenAI(model="gpt-4o-mini", api_key=api_key)

- OpenAI가 아닌 Gemini 로드 방법.
```python
from langchain_google_genai import ChatGoogleGenerativeAI

model = ChatGoogleGenerativeAI(model="gemini-pro")
```

## *문서 로드*

- Langchain의 `PyPDFLoader` 를 이용.

In [2]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("한권으로 끝내는 주식과 세금.pdf")

# 페이지 별 문서 로드
docs = loader.load()

## *문서 청크로 나누기*

1. `CharacterTextSplitter`

- 텍스트를 일정한 문자 단위로 나누는 기본적인 청크 분할 도구.
- `chunk_size`, `chunk_overlap`을 기반으로 단순히 분할함.
```python
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = text_splitter.split_documents(docs)
```

2. `RecursiveCharacterTextSplitter` : *재귀적 파편화 기법*

- 위 방법보다 더 정교한 방식으로 텍스트를 청크로 나눈다.  
- 여러 구분자 목록을 순차적으로 사용해 우선 순위 높은 구분자를 사용해 텍스트를 나눔. 기본 우선 순위는 `['\n\n', '\n', ' ', '']`이다.
- `chunk_size`를 만족할 때까지 구분자로 텍스트를 나눈 뒤, 이 과정을 반복.
- 만약 구분자를 모두 적용해도 `chunk_size`를 만족 못하면 `''`로 단순히 자름.
```python
from langchain.text_splitter import RecursiveCharacterTextSplitter

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=10,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(docs)
```

### *파라미터*

- `separator` : 청크 분할에 사용할 구분자. 예로 \n\n은 문단 단위로 청크를 나눈다는 뜻. 설정을 하지 않으면 chunk_size 기준으로 단순히 분리함.
- `chunk_size` : 최대 청크의 길이. n글자.
- `chunk_overlap` : 청크 사이의 중복 길이, 첫 청크, 두 청크 사이 n글자의 중복 텍스트를 가짐.
- `length_function` : 각 청크를 len 함수로 텍스트 길이 계산.
- `is_separator_regex` : 분리할 때 구분자를 정규 표현식으로 사용할지 여부를 나타내는 값. False면 일반 문자열 구분자를 그대로 사용.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 차례의 내용들이 큰 의미가 없다고 판단해 차례 페이지는 과감히 제외.
filtered_docs = [doc for i, doc in enumerate(docs) if not (3 <= i <= 13)]

recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)

splits = recursive_text_splitter.split_documents(filtered_docs)

for i, chunk in enumerate(splits[:10]):
    print(f"Chunk {i+1}: {chunk}")

Chunk 1: page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  
국민들의 주식시장에 대한 관심이 크게 증가하였습니다.
최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  
관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  
쉽게 찾아보기 어려운 게 현실입니다.
이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  
이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.
이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  
부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, 
인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 
발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.
모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를
기대합니다.
2024.  5
국세청 자산과세국장
머리말' metadata={'source': '한권으로 끝내는 주식과 세금.pdf', 'page': 2}
Chunk 2: page_content='3
조금 더 알아보기
주식이란 주식회사를 설립하거나 사업확장 등을 위해 필요한 자금을 조달할 때 투자자 
에게 자금을 보탠 대가로 발행해 주는 증서로서 주식회사의 소유지분을 표시하는 단위 
입니다.
주식은 작은 금액의 단위 (1주당 100원 이상) 로 발행되는데, 이는 많은 사람이 자신의  
사정에 맞게 투자할 수 있도록 하기 위한 것입니다. 결국 주식회사가 수많은 사람들로부터 
자금을 제공받아 그것을 원천으로 기업활동을 계속할 수 있도록 해 주는 것이죠.
주식을 소유한 사람을 주주라고 하고, 주주는 회사의 자본금 중 자신이 출자한 금액  
만큼 회사의 주인이 되며, 출자지분에 비례하여 배당을 받게 됩니다. 회사 측면에서는 
주

## *벡터 임베딩 생성*

- OpneAI의 Ada 모델은 성능이 우수하고 비용 효율적인 임베딩 모델. 높은 차원의 벡터 생성, 텍스트간 유사도 비교에 효과적.
- Gemini 모델일 경우, `GoogleGenerativeAIEmbeddings` 이용.

```python
from langchain_google_genai import GoogleGenerativeAIEmbeddings

# gemini의 임베딩 모델
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
```

In [4]:
from langchain_openai import OpenAIEmbeddings

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

## *Vector Store 생성*

- 앞서 만든 벡터 임베딩, 청크된 문서를 활용해 FAISS 벡터 스토어 생성.

In [ ]:
import faiss
from langchain_community.vectorstores import FAISS

vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)

## *FAISS를 Retriever로 변환*

- RAG 체인에서 사용할 수 있도록 FAISS -> retriever(검색기) 변환.

- `search_type = "similarity"` : 검색의 유형, 유사도 기반으로 설정. 쿼리와 벡터 공간에서 유사한 문서들을 검색.
- `search_kwargs{"k":1}` : 검색 관련 추가 매개변수 지정할 수 있는 딕셔너리. k는 검색 결과로 반환할 가장 유사한 문서의 개수 의미. 고로, 쿼리에 대해 가장 유사한 하나의 문서만 반환하도록 설정.

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 1})

## *프롬프트 템플릿 정의*

- `ChatPromptTemplate.from_messages()`는 메시지 목록을 인자로 받음. 각 메시지는 발신자 역할과 텍스트로 구성.
- `contextual_prompt` :  
    - 시스템 메시지 : 주어진 맥락 정보만 사용해 답변하라.
    - 유저 메시지의 `{context}, {question}`는 자리 표시자. 실제 응답 생성 시 실제 맥락 정보와 질문 내용이 이 위치에 삽입.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
            
# 프롬프트 템플릿 정의
contextual_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the question using only the following context."),
    ("user", "Context: {context}\\n\\nQuestion: {question}")
])

## RAG 체인 구성

- LangChain model과 Prompt template를 연결해 RAG Chains 구성.
- `DebugPassThrough class` 활용해 각 단계에서 생성되는 출력물을 확인.
    - `invoke 메서드`를 오버라이드하여 상위 클래스의 `invoke 메서드` 결과를 `output`에 저장. 출력하고 반환.

- 문서리스트에서 텍스트를 추출해 하나의 문자열로 변환하는 단계(`ContextToText`)도 추가. 
    - `invoke 메서드`는 `inputs 딕셔너리`로부터 context와 question을 가져옴.
    - context는 문서 리스트 형태로 전달, 각 문서의 page_content 추출해서 하나의 문자열로 결합.
    - return값 형태로 이후 단계에서 프롬프트에 전달.

- `rag_chain_debug` : RAG 체인 각 단계 정의, `DebugPassThrough`를 각 주요 단계에 추가해 중간 결과 디버깅.
    - 각 단계는 파이프 연산자 통해 연결.

- *구성 정리*
    - `retriever` - 사용자 질문과 관련된 문서 리스트를 검색해 제공하는 역할
    - `DebugPassThrough()` - question을 그대로 전달하며 출력해 사용자 질문이 정확히 전달되고 있는지 확인.
    - `ContextToText()` - retriever가 제공한 문서 리스트를 텍스트로 변환해 프롬프트 입력 형태에 맞춰 전달.
    - `contextual_prompt` - 템플릿에서 context, question 바탕으로 최종 프롬프트 생성.
    - `model` - 최종 생성 프롬프트 바탕 답변 생성.

In [ ]:
class DebugPassThrough(RunnablePassthrough):
    def invoke(self, *args, **kwargs):
        output = super().invoke(*args, **kwargs)
        print("Debug Output:", output)
        return output
    
# 문서 리스트를 텍스트로 변환하는 단계 추가
class ContextToText(RunnablePassthrough):
    def invoke(self, inputs, config=None, **kwargs):  # config 인수 추가
        # context의 각 문서를 문자열로 결합
        context_text = "\n".join([doc.page_content for doc in inputs["context"]])
        return {"context": context_text, "question": inputs["question"]}

# RAG 체인에서 각 단계마다 DebugPassThrough 추가
rag_chain_debug = {
    "context": retriever,   # 컨텍스트를 가져오는 retriever
    "question": DebugPassThrough()  # 사용자 질문이 그대로 전달되는지 확인하는 passthrough
}  | DebugPassThrough() | ContextToText() | contextual_prompt | model

## Chat Bot 구동 확인

1. 기본 Chat GPT(*gpt-4o-mini*)에 입력한 질문 :

In [12]:
query = "국내 상장법인 주식을 보유한 개인 투자자는 몇명이야?"

response_basic = model.invoke([HumanMessage(content=query)])
print(response_basic.content)

국내 상장법인 주식을 보유한 개인 투자자의 수는 변동성이 크고 주기적으로 업데이트되는 정보입니다. 2023년 기준으로 한국의 증권 시장에서는 개인 투자자 수가 약 14~15 million명에 달하는 것으로 알려져 있습니다. 하지만 이 숫자는 시장 상황이나 투자 트렌드에 따라 변동할 수 있으므로, 최신 통계는 한국거래소나 금융감독원 등의 공식 자료를 참조하는 것이 좋습니다.


2. 구성된 RAG 체인을 통해 질문 :

In [13]:
response = rag_chain_debug.invoke(query)
print("Final Response:")
print(response.content)

Debug Output: 국내 상장법인 주식을 보유한 개인 투자자는 몇명이야?
Debug Output: {'context': [Document(metadata={'source': '한권으로 끝내는 주식과 세금.pdf', 'page': 2}, page_content='지난해 말 국내 상장법인 주식을 보유한 개인 투자자가 1,400만명을 넘어서는 등  \n국민들의 주식시장에 대한 관심이 크게 증가하였습니다.\n최근 일반 국민들의 주식투자에 대한 관심이 크게 증가했음에도 불구하고, 주식 투자  \n관련 세금문제 등 궁금한 사항에 대하여 도움을 줄 수 있는 안내책자 등은 시중에서  \n쉽게 찾아보기 어려운 게 현실입니다.\n이에 국세청에서는 주식 관련 각종 세금에 대한 납세자들의 이해를 높이고 납세의무  \n이행에 도움이 될 수 있도록 「주식과 세금」 책자를 처음으로 제작·발간하게 되었습니다.\n이번에 새롭게 출간하는 ‘주식과 세금’ 책자는 주식거래의 기초상식과 주식의 취득  \n부터 보유 및 처분시까지 단계별 세금문제를 총 76개의 문답형식으로 구성하는 한편, \n인포그래픽 등을 적극 활용하여 가독성을 제고하였으며, 구체적인 절세 꿀팁 및 자주 \n발생하는 실수 사례 등을 추가하여 활용성도 강화하였습니다.\n모쪼록, 이 책자가 주식등 관련 납세자들의 성실한 납세의무 이행에 기여할 수 있기를\n기대합니다.\n2024.  5\n국세청 자산과세국장\n머리말')], 'question': '국내 상장법인 주식을 보유한 개인 투자자는 몇명이야?'}
Final Response:
1,400만명입니다.


3. RAG가 필요한 이유 :

- 정확한 정보를 제공한다는 것에 RAG가 필요한 이유라고 생각됨. RAG체인을 통하지 않고 일반 LLM에 답변을 요구했을 때, 최신 정보 업데이트를 감안한 불확실성을 띄고 있지만, RAG 체인을 통해 답변을 요구했더니 근거가 된 단락의 내용을 필두로 확실하게 답변을 제시하고 있음. 신뢰성 높은 답변을 생성할 수 있도록 지원하는 것이 RAG의 필요성이라고 생각함.